In [63]:
# !python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 46.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [64]:
import os
import re
import spacy
import awswrangler as wr
import pandas as pd

## Carga de datos

In [32]:
df_hogar_mania = wr.s3.read_parquet(path='s3://raw-recipe-data-bucket/hogar_mania')
df_antena3 = wr.s3.read_parquet(path='s3://raw-recipe-data-bucket/antena3')
df_saber_vivir = wr.s3.read_parquet(path='s3://raw-recipe-data-bucket/saber_vivir')
df_el_mundo = wr.s3.read_parquet(path='s3://raw-recipe-data-bucket/el_mundo')

### Hogar Mania

In [33]:
categorias_hogar_mania = df_hogar_mania['categoria'].unique()
categorias_hogar_mania

<StringArray>
[             'postres',           'aperitivos',   'ensaladas-verduras',
     'arroces-cereales',        'pastas-pizzas',         'sopas-cremas',
            'legumbres',               'carnes',    'pescados-mariscos',
         'setas-hongos',               'huevos',               'salsas',
            'desayunos',          'recetas-pan',              'bebidas',
             'cocteles', 'cocina-internacional']
Length: 17, dtype: string

In [34]:
df_hogar_mania.shape

(7464, 5)

In [35]:
df_hogar_mania.dropna(subset=["ingredientes"], inplace=True)

In [36]:
df_hogar_mania.shape

(7228, 5)

In [37]:
df_hogar_mania['total_ingredientes'] = df_hogar_mania['ingredientes'].apply(len)
df_hogar_mania[df_hogar_mania['total_ingredientes'] == 0].shape

(18, 6)

In [38]:
df_hogar_mania = df_hogar_mania[df_hogar_mania['total_ingredientes'] != 0]

In [39]:
df_hogar_mania[df_hogar_mania.duplicated(subset=['link'], keep=False)]['categoria'].unique()

<StringArray>
[             'postres',           'aperitivos',   'ensaladas-verduras',
     'arroces-cereales',        'pastas-pizzas',         'sopas-cremas',
            'legumbres',               'carnes',    'pescados-mariscos',
               'huevos',            'desayunos',          'recetas-pan',
              'bebidas',             'cocteles', 'cocina-internacional']
Length: 15, dtype: string

In [40]:
df_hogar_mania_double = df_hogar_mania[df_hogar_mania.duplicated(subset=['link'], keep=False)]
df_hogar_mania_double.groupby(by='link')['categoria'].apply(list)

link
https://www.hogarmania.com//cocina/recetas/aperitivos/desayuno-real-cereal-37095.html                                        [aperitivos, desayunos]
https://www.hogarmania.com//cocina/recetas/aperitivos/jocoque-10518.html                                          [aperitivos, cocina-internacional]
https://www.hogarmania.com//cocina/recetas/aperitivos/molletes-mexicana-36967.html                                           [aperitivos, desayunos]
https://www.hogarmania.com//cocina/recetas/aperitivos/zumo-papaya-naranja-pomelo-25672.html                                    [aperitivos, bebidas]
https://www.hogarmania.com//cocina/recetas/arroces-cereales/arroz-colores-salsa-agridulce-asia-2684.html    [arroces-cereales, cocina-internacional]
                                                                                                                              ...                   
https://www.hogarmania.com//cocina/recetas/recetas-pan/bagels-rosquillas-18773.html                  

In [41]:
df_hogar_mania.drop_duplicates(subset=['link'], keep='first', inplace=True)

In [42]:
df_hogar_mania.shape

(7056, 6)

In [ ]:
def extract_words(text):
    words = re.findall(r'\b[a-zA-Z]+\b', text)
    return ' '.join(words)

In [60]:
df_hogar_mania['titulo_link'] = (
    df_hogar_mania['link']
    .apply(os.path.basename)
    .apply(os.path.splitext)
    .apply(lambda text: text[0])
    .apply(extract_words)
)
df_hogar_mania[['titulo', 'titulo_link']]

,titulo,titulo_link
0,"Buñuelos de viento fáciles, la receta tradicio...",bunuelos
1,Corona de hojaldre de Navidad,corona navidad
2,Cafés de Navidad: Gingerbread Latte y Pumpkin ...,cafes navidad gingerbread pumpkin spice latte
4,Mazapanes de Navidad,mazapanes navidad
6,"Churros en freidora de aire, ¡más fácil imposi...",churros freidora aire
...,...,...
7384,"Tarta fría de limón de Eva Arguiñano, la más c...",tarta fria limon
7385,Receta de mochis japoneses caseros,receta mochis japoneses caseros
7386,"Receta de helado de pistacho, paso a paso",helado pistacho casero
7387,Cómo hacer gachas dulces con los pasos de Karl...,gachas dulces


In [65]:
nlp = spacy.load('es_core_news_sm')

In [216]:
def extract_ingredients(text):
    doc = nlp(text.lower())
    unnecessary_bow = [
        "gr",
        "g",
        "kg",
        "ml",
        "l",
        "oz",
        "lb",
        "gr.",
        "g.",
        "kg.",
        "ml.",
        "l.",
        "oz.",
        "lb.",
        "kilo",
        "litro",
        "gramo",
        "vaso",
        "vasos",
        "taza",
        "tazón",
        "pizca",
        "cucharada",
        "cucharadas",
        "cucharadita",
        "ralladura",
    ]
    admitted_class_words = ["NOUN", "PROPN", "PRON", "ADJ"] # 220 gr. azúcar | 1 cebolla => lo identifica como VERB ?!
    filtered_tokens = [
        token.text
        for token in doc
        if not token.like_num
        and not token.is_stop
        and not token.is_punct
        and token.pos_ in admitted_class_words
        and token.text not in unnecessary_bow
    ]
    return " ".join(filtered_tokens) # TODO do not join empty tokens

In [177]:
df_hogar_mania['ingredientes_limpios'] = df_hogar_mania['ingredientes'].apply(lambda ingrs: [extract_ingredients(ingr) for ingr in ingrs])

0       [harina, mantequilla, agua, azúcar, huevos, li...
1       [láminas hojaldre, chocolate negro, mantequill...
2       [azúcar moreno suave, jengibre molido, nuez mo...
4       [almendra molida, leche condensada, limón, , a...
6       [churros congelados, aceite oliva girasol, azú...
                              ...                        
7384                      [galletas, mantequilla fundida]
7385    [harina arroz glutinoso, agua, azúcar recetas ...
7386    [g. pasta pistachos comercial casera, g. pista...
7387    [leche, harina, azúcar, semillas anís, , limón...
7388    [variedad chuches gusto botes kilo euros, palo...
Name: ingredientes, Length: 7056, dtype: object

### Antena3

In [208]:
categorias_antena3 = df_antena3['categoria'].unique()
categorias_antena3

<StringArray>
['ensaladas', 'entrantes',   'arroces',   'pescado',    'carnes',   'postres',
    'guisos',    'huevos',     'pasta']
Length: 9, dtype: string

In [196]:
df_antena3.shape

(2022, 5)

In [197]:
df_antena3['total_ingredientes'] = df_antena3['ingredientes'].apply(len)

In [198]:
df_antena3 = df_antena3[df_antena3['total_ingredientes'] != 0]
df_antena3 = df_antena3[(df_antena3['elaboracion'].str.len() != 0)]

In [199]:
df_antena3.shape

(1761, 6)

In [202]:
df_antena3[df_antena3['total_ingredientes'] > 14].shape

(428, 6)

In [203]:
df_antena3['ingredientes'] = (
    df_antena3
    ['ingredientes']
    .apply(lambda ingrs: ingrs[:14] if len(ingrs) > 14 else ingrs)
)

In [204]:
df_antena3['total_ingredientes'] = df_antena3['ingredientes'].apply(len)
df_antena3['total_ingredientes'].describe()

count    1761.000000
mean       10.781374
std         2.838336
min         2.000000
25%         8.000000
50%        11.000000
75%        14.000000
max        14.000000
Name: total_ingredientes, dtype: float64

In [205]:
df_antena3['titulo_link'] = (
    df_antena3['link']
    .apply(os.path.basename)
    .apply(os.path.splitext)
    .apply(lambda text: text[0])
    .apply(extract_words)
)
df_antena3[['titulo', 'titulo_link']]

,titulo,titulo_link
0,"Receta de alcachofas en salsa, de Karlos Argui...",alcachofas
1,"Una receta sencilla, sabrosa y rápida de Joseb...",ensaladilla
2,"Brócoli con champiñones y patatas, de Arguiñan...",brocoli champinones
3,"Berenjenas a la crema, de Karlos Arguiñano: ""U...",berenjenas
4,Espárragos a la plancha con vinagreta de fresa...,esparragos plancha vinagreta fresas
...,...,...
2017,Arroz con verduras y conejo al azafrán,video arroz con verduras y
2018,La deliciosa receta de canelones de cocido de ...,la deliciosa receta de canelones de cocido de ...
2019,Macarrones con crema de calabaza y queso,macarrones crema calabaza
2020,Canelones de verduras y jamón cocido,canelones de verduras y jamon


In [206]:
df_antena3['ingredientes_limpios'] = df_antena3['ingredientes'].apply(lambda ingrs: [extract_ingredients(ingr) for ingr in ingrs])
df_antena3[['ingredientes','ingredientes_limpios']]

,ingredientes,ingredientes_limpios
0,"[8 huevos de codorniz, 1 cebolla, 2 dientes de...","[huevos, , dientes ajo, txakoli, caldo verdura..."
1,"[3 patatas, 3 zanahorias, 3 huevos, 4 lonchas ...","[patatas, zanahorias, huevos, lonchas queso ha..."
2,"[6 champiñones, 2 patatas, 4 dientes de ajo, 1...","[champiñones, patatas, dientes ajo, jamón serr..."
3,"[1 cebolla pequeña, 4 dientes de ajo, ½ l de l...","[cebolla pequeña, dientes ajo, leche, caldo ca..."
4,"[4-6 fresas (dependiendo del tamaño), ½ aguaca...","[fresas tamaño, aguacate, lima, aceite oliva v..."
...,...,...
2017,"[300 g de arroz, 2 muslos de conejo, 1 cebolle...","[arroz, muslos conejo, , espárragos verdes, di..."
2018,"[24 láminas de canelón, 1 muslo de gallina (co...","[láminas canelón, muslo gallina cocido, trozos..."
2019,"[300 g de macarrones, 400 g de calabaza (potim...","[macarrones, calabaza potimarrón, , puerro, qu..."
2020,"[20 láminas de canelones, 200 g de espinacas, ...","[láminas canelones, espinacas, , zanahorias, d..."


### Saber Vivir

In [213]:
categorias_saber_vivir = df_saber_vivir['categoria'].unique()
categorias_saber_vivir =[categoria.lower() for categoria in categorias_saber_vivir]
categorias_saber_vivir

['caldos y cremas',
 'postres y dulces',
 'celebraciones',
 'pasteles salados',
 'pastas y pizzas',
 'pescados y mariscos',
 'ensaladas y verduras',
 'legumbres',
 'carnes',
 'arroces']

In [207]:
df_saber_vivir.shape

(461, 5)

In [214]:
df_saber_vivir['total_ingredientes'] = df_saber_vivir['ingredientes'].apply(len)

In [215]:
df_saber_vivir['titulo_link'] = (
    df_saber_vivir['link']
    .apply(os.path.basename)
    .apply(os.path.splitext)
    .apply(lambda text: text[0])
    .apply(extract_words)
)
df_saber_vivir[['titulo', 'titulo_link']]

,titulo,titulo_link
0,Crema de calabaza,crema de calabaza
1,Crema de champiñones y queso,crema de champinones y queso
2,Crema de coles de Bruselas con jamón,crema de coles de bruselas con jamon
3,Crema de espárragos con caviar,crema de esparragos con caviar
4,Crema de espinacas,crema de espinacas
...,...,...
456,Crema castellana con huevo,crema castellana con huevo
457,Crema catalana a los tres cítricos,crema catalana a los tres citricos
458,Crema catalana con hierbaluisa,crema catalana con hierbaluisa
459,"Crema de boletus, castañas y berenjenas",crema de boletus castanas y berenjenas


In [217]:
df_saber_vivir['ingredientes_limpios'] = df_saber_vivir['ingredientes'].apply(lambda ingrs: [extract_ingredients(ingr) for ingr in ingrs])
df_saber_vivir[['ingredientes','ingredientes_limpios']]

,ingredientes,ingredientes_limpios
0,"[1 puerro pequeño, Media cebolla, 1 zanahoria,...","[puerro pequeño, cebolla, , tomate, patatas, c..."
1,"[600 gr. champiñones, 4 vasos leche, 2 vaso de...","[champiñones, leche, caldo ave, quesitos, pata..."
2,"[12 coles de Bruselas, 5 patatas medianas, Ace...","[coles bruselas, patatas medianas, aceite oliv..."
3,"[1 manojo de espárragos verdes, 1 litro de cal...","[manojo espárragos verdes, caldo verduras, lat..."
4,"[2 dientes de ajo picados, 1 vaso de arroz coc...","[dientes ajo picados, arroz cocido, caldo, pic..."
...,...,...
456,"[8 dientes de ajo, 4 huevos, 100 gr. pan, 100 ...","[dientes ajo, huevos, pan, chorizo, caldo ave,..."
457,"[1 litro leche, 220 gr. azúcar, 40 gr. harina ...","[leche, , harina maíz refinada, yemas huevo, l..."
458,"[2 cucharadas de azúcar, 2 cucharadas de nata,...","[azúcar, nata, mantequilla, chocolate negro fo..."
459,"[100 gr. boletus, 100 gr. setas shiitake, 80 g...","[boletus, setas shiitake, castañas, , brandy, ..."


### El Mundo

In [222]:
categorias_el_mundo = df_el_mundo['categoria'].unique()
categorias_el_mundo

<StringArray>
[                 'postres',                    'pasta',
                    'arroz',                    'pollo',
             'sopas-caldos',                'bizcochos',
                    'carne',                   'guisos',
                'legumbres',                 'verduras',
                   'salsas',              'masas-panes',
                   'cremas', 'patatas-huevos-tortillas',
       'aperitivos-faciles',                 'pescados',
         'recetas-mariscos',             'vegetarianas',
            'sanas-faciles',               'orientales']
Length: 20, dtype: string

In [223]:
df_el_mundo.shape

(388, 5)